# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     |Santiago Escutia Ríos  |
| **Fecha**      | 8/2/2026  |
| **Expediente** |  757839 |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

¿Qué puedes observar al ajustar los valores de `alpha`? 

Compara los resultados de los coeficientes utilizando valores diferentes de $\alpha$ y los $R^2$ resultantes.



1. Sin penalización (Regresión normal)

In [37]:
import pandas as pd
from sklearn.linear_model import LinearRegression

df = pd.read_csv('Advertising.csv', index_col=0)
X = df[['TV', 'radio', 'newspaper']]
y = df['sales']

modelo_lr = LinearRegression()
modelo_lr.fit(X, y)

print(f"Coeficientes (Sin penalización): {modelo_lr.coef_}")
print(f"R^2 (Sin penalización): {modelo_lr.score(X, y):.4f}")

Coeficientes (Sin penalización): [ 0.04576465  0.18853002 -0.00103749]
R^2 (Sin penalización): 0.8972


2. Con penalización L2 (Ridge)

In [38]:
from sklearn.linear_model import Ridge

modelo_ridge = Ridge(alpha=100)
modelo_ridge.fit(X, y)

print(f"Coeficientes (Ridge L2): {modelo_ridge.coef_}")
print(f"R^2 (Ridge L2): {modelo_ridge.score(X, y):.4f}")

Coeficientes (Ridge L2): [ 0.04576446  0.18803935 -0.00091803]
R^2 (Ridge L2): 0.8972


Ahora con alphas más altos.

In [39]:
from sklearn.linear_model import Ridge

modelo_ridge = Ridge(alpha=1000000)
modelo_ridge.fit(X, y)

print(f"Coeficientes (Ridge L2): {modelo_ridge.coef_}")
print(f"R^2 (Ridge L2): {modelo_ridge.score(X, y):.4f}")

Coeficientes (Ridge L2): [0.02818664 0.00804639 0.0040067 ]
R^2 (Ridge L2): 0.5414


Probando valores alpha diferentes se ve que en valores de 1 a 1,000 no cambia en casi nada, lo que indica que son valores buenos, pero al introducir valores mas grande como 10,000-100,000 el R2 empieza a empeorar considerablemente y los coeficientes se empiezan a acercar mucho a cero (Valores así de altos no hay porque usarse).

3. Con penalización L1 (Lasso)

In [40]:
from sklearn.linear_model import Lasso

modelo_lasso = Lasso(alpha=1)
modelo_lasso.fit(X, y)

print(f"Coeficientes (Lasso L1): {modelo_lasso.coef_}")
print(f"R^2 (Lasso L1): {modelo_lasso.score(X, y):.4f}")

Coeficientes (Lasso L1): [0.04566142 0.1834644  0.        ]
R^2 (Lasso L1): 0.8970


Ahora probando con alphas más altos

In [41]:
from sklearn.linear_model import Lasso

modelo_lasso = Lasso(alpha=100)
modelo_lasso.fit(X, y)

print(f"Coeficientes (Lasso L1): {modelo_lasso.coef_}")
print(f"R^2 (Lasso L1): {modelo_lasso.score(X, y):.4f}")

Coeficientes (Lasso L1): [0.03390169 0.         0.        ]
R^2 (Lasso L1): 0.5615


Conforme se aumenta el valor de alpha en Lasso el R2 empieza a empeorar muy rápido (incluso más rápido que en Ridge para valores no tan altos), además de que entre más alto el alpha los coefieicntes eventualmente llegan a cero, algo que que es característico de Lasso. Curiosamente se ve que con alpha=1 ya directamente newspaper da 0 lo que confirma que no es un medio significante para las ventas.